In [54]:
import yfinance 
import plotly.graph_objects as go
import pandas as pd
import matplotlib.pyplot as plt

In [22]:
data = yfinance.download("BTC-USD", period="5y")

C:\Users\mateu\AppData\Local\Temp\ipykernel_18888\2422031220.py:1: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed


In [23]:
FONT_FAMILY = 'ui-sans-serif,-apple-system,system-ui,Segoe UI,Helvetica,Apple Color Emoji,Arial,sans-serif,Segoe UI Emoji,Segoe UI Symbol'

In [24]:
import numpy as np


def plot_chat_Candlestick(data, symbol='BTC-USD', scale=None):

    if scale == 'log':
        data = data.apply(lambda x: np.log1p(x))
        
    elif scale == 'sqrt':
        data = data.apply(lambda x: np.sqrt(x))

    
    fig = go.Figure([
        go.Candlestick(
            x=data.index,
            open=data['Open'].values.flatten(),   # transforma em 1D
            high=data['High'].values.flatten(),
            low=data['Low'].values.flatten(),
            close=data['Close'].values.flatten(),
        )
    ])
    fig.update_traces(
        hoverlabel = dict(
            font = dict(
                size = 16
            )
        )
    )
    fig.update_layout(
        title = dict(
            text = f'Candlestick {symbol}',
            font = dict(
                family = FONT_FAMILY,
                weight = 'bold'
            )
        ),
        xaxis = dict(
            title = dict(
                text = 'Date'
            )
        ),
        font = dict(
            family = FONT_FAMILY
        ),
        margin = dict(
            l = 10,
            r = 10,
            t = 40,
            b = 40
        ),
        template = 'simple_white',
        hovermode = 'x unified',
        xaxis_rangeslider_visible=False

    )
    return fig

In [25]:
plot_chat_Candlestick(data).show()

In [26]:
def volatility(data):
    # daily Volatility : Close - Open
    
    data['volatility'] = data['High'] - data['Low']
 
    fig = go.Figure([
        go.Scatter(
            x=data.index, 
            y=data['volatility'], 
            mode='lines'
        )
    ])

    title_text = 'Daily Volatility'
    fig.update_layout(
        title = dict(
            text = title_text,
            font = dict(
                family = FONT_FAMILY,
                weight = 'bold'
            ) 
        ),
        xaxis = dict(
            title = dict(
                text = 'Date'
            )
        ),
        yaxis = dict(
            title = dict(
                text = 'Volatility'
            )
        ),
        font = dict(
            family = FONT_FAMILY,
        ),
        template = 'simple_white'
    )
    return fig

In [27]:
volatility(data).show()

In [28]:
def return_rate(data, window=0):

    if window == 0:
        data['return_rate'] = data['Close'] - data['Open']
    else:
        data['return_rate'] = data['Close'].pct_change(window)
        data = data.dropna()

    fig = go.Figure([
        go.Scatter(
            x=data.index, 
            y=data['return_rate'], 
            mode='lines'
        )
    ])

    fig.update_traces(
        hoverlabel = dict(
            bgcolor = 'white',
            font = dict(
                size = 16
            )
        ),
        hovertemplate = '%{x} <br> Return Rate : %{y}<extra></extra>'
    )
    
    title_text = 'Return Rate, the last '+ str(window) + ' days'

    fig.add_hline(
        y=0, 
        line_width=2,
        line_dash ='solid', 
        line_color="black"
    )
    fig.update_layout(
        title = dict(
            text = title_text,
            font = dict(
                family = FONT_FAMILY,
                weight = 'bold'
            ) 
        ),
        xaxis = dict(
            title = dict(
                text = 'Date'
            )
        ),
        yaxis = dict(
            title = dict(
                text = 'Return Rate'
            )
        ),
        
        font = dict(
            family = FONT_FAMILY,
        ),
        template = 'simple_white',
        hovermode = 'x'
    )
    return fig

In [29]:
return_rate(data, 40).show()

In [ ]:
import statsmodels.api as sm
from plotly.subplots import make_subplots

def decompose_time_series(data, model='additive', freq=None):
    # model = 'additive' or 'multiplicative'
    # freq = None (let statsmodels decide) or integer (number of periods in a season)
    
    result = sm.tsa.seasonal_decompose(data['Close'].values.ravel(), model=model, period=freq)
    
    fig = go.Figure()
    # Transformar em DataFrame
    df_decomp = pd.DataFrame({
        "observed": result.observed,
        "trend": result.trend,
        "seasonal": result.seasonal,
        "resid": result.resid
    }, index=data.index)

    # Criar subplots com Plotly
    fig = make_subplots(rows=4, cols=1, shared_xaxes=True,
                        subplot_titles=("Observed", "Trend", "Seasonal", "Residuals"))

    fig.add_trace(
        go.Scatter(
            x=df_decomp.index, 
            y=df_decomp["observed"], 
            name="Observed"
        ), row=1, col=1
    )
    fig.add_trace(
        go.Scatter(
            x=df_decomp.index, 
            y=df_decomp["trend"], 
            name="Trend"
        ), row=2, col=1
    )
    fig.add_trace(
        go.Scatter(
            x=df_decomp.index, 
            y=df_decomp["seasonal"], 
            name="Seasonal"
            ), row=3, col=1
        )
    
    fig.add_trace(
        go.Scatter(
            x=df_decomp.index, 
            y=df_decomp["resid"], 
            name="Residuals"
        ), row=4, col=1
    )

    fig.update_layout(height=900, width=1000, title_text="Time Series Decomposition (Plotly)")
    fig.update_layout(
        title = dict(
            text = 'Decomposition of the time series',
            font = dict(
                family = FONT_FAMILY,
                weight = 'bold'
            )
        ),
        font = dict(
            family = FONT_FAMILY
        ),
        template = 'simple_white',
        dragmode=None
    )
    return fig

In [84]:
decompose_time_series(data, model='multiplicative', freq=5).show()